<a href="https://colab.research.google.com/github/OscarLoOscar/python_Data_Science/blob/main/GPT_4o_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import shlex
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import drive
import numpy as np
import tensorflow as tf
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
import re
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Precision
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [30]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **拎Big Data**



In [41]:
class HKJCWebScraper:
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--remote-debugging-port=9222")
        self.browser = webdriver.Chrome(options=chrome_options)
        self.base_url = 'https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate='

    def get_dates(self):
        self.browser.get(self.base_url + '2024/05/29')  # Example date to access the page
        try:
            # Wait for the select element to be present
            select_element = WebDriverWait(self.browser, 20).until(
                EC.presence_of_element_located((By.ID, "selectId"))
            )
            options = WebDriverWait(self.browser, 20).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "option"))
            )
            dates = [option.get_attribute("value") for option in options]
            return dates
        except Exception as e:
            print(f"An error occurred while fetching dates: {e}")
            return []

        # Extract data from performance table
        performance_data = []
        performance_table = soup.find('table', class_='f_tac table_bd draggable')
        if performance_table:
            rows = performance_table.find_all('tr')
            for row in rows:
                performance_data.append([cell.get_text(strip=True) for cell in row.find_all('td')])

       # Extract data from dividend_tab
        dividend_data = []
        dividend_table = soup.find('table', class_='table_bd f_tac f_fs13 f_fl')
        if dividend_table:
            rows = dividend_table.find_all('tr')
            for row in rows:
                dividend_data.append([cell.get_text(strip=True) for cell in row.find_all('td')])

        return race_tab_data, performance_data, dividend_data
       # return race_tab_data, performance_data

    def close(self):
        self.browser.quit()

scraper = HKJCWebScraper()


In [ ]:
all_data = []
dates = scraper.get_dates()
for date in dates:
    print(f"Scraping data for {date}")
    try:
        data = scraper.scrape_data_for_date(date)
        all_data.append({date: data})
    except Exception as e:
        print(f"Failed to scrape data for {date}: {e}")

# file_path = '/content/drive/My Drive/horse_data2.xlsx'
# all_data.to_excel(file_path, index=False)
def parse_data(all_data):
    data_list = []
    for race in all_data:
        for date, (meta, race_results) in race.items():
            if not race_results:  # 檢查 race_results 是否為空
                print(f"No race results for {date}")
                continue

            columns = race_results[0]
            for result in race_results[1:]:
                if len(result) != len(columns):
                    print(f"Mismatch in columns and data for {date}: {result}")
                    continue

                race_dict = {col: val for col, val in zip(columns, result)}
                race_dict['date'] = date
                data_list.append(race_dict)

    return pd.DataFrame(data_list)

df = parse_data(all_data)

# 保存為Excel文件
file_path = '/content/drive/My Drive/horse_data.xlsx'
df.to_excel(file_path, index=False)



In [25]:
import requests
# 定义目标 URL
base_url = "https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate="

def get_dates():
    response = requests.get(base_url + '2024/05/29')
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    select_element = soup.find('select', {'id': 'selectId'})
    options = select_element.find_all('option')
    dates = [option.get('value') for option in options]
    return dates

def scrape_data_for_date(date):
    response = requests.get(base_url + date)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    # Check if the request was successful
    if response.status_code == 200:
        # Find and print the race date and location
        race_date_location = soup.find('p', class_='f_clear')
        if race_date_location:
            race_date_location_text = race_date_location.text.strip()
        else:
            race_date_location_text = "N/A"

        # Find and print the race class and distance
        race_class_distance = soup.find('td', style='width: 385px;')
        if race_class_distance:
            race_class_distance_text = race_class_distance.text.strip()
        else:
            race_class_distance_text = "N/A"

        # Find and print the field condition
        field_condition_label = soup.find('td', style='width: 110px;', string='場地狀況 :')
        if field_condition_label:
            field_condition = field_condition_label.find_next_sibling('td')
            if field_condition:
                field_condition_text = field_condition.text.strip()
            else:
                field_condition_text = "N/A"
        else:
            field_condition_text = "N/A"

        # Find and print the track type
        track_type_label = soup.find('td', string='賽道 :')
        if track_type_label:
            track_type = track_type_label.find_next_sibling('td')
            if track_type:
                track_type_text = track_type.text.strip()
            else:
                track_type_text = "N/A"
        else:
            track_type_text = "N/A"

        return {
            "賽事日期和地點": race_date_location_text,
            "賽事等級和距離": race_class_distance_text,
            "場地狀況": field_condition_text,
            "賽道": track_type_text
        }
    else:
        print(f"Failed to retrieve the website content for date: {date}")
        return None

all_data = []
dates = get_dates()
for date in dates:
    print(f"Scraping data for {date}")
    try:
        data = scrape_data_for_date(date)
        if data:
            all_data.append({date: data})
    except Exception as e:
        print(f"Failed to scrape data for {date}: {e}")

def parse_data(all_data):
    data_list = []
    for race in all_data:
        for date, race_results in race.items():
            race_results['date'] = date
            data_list.append(race_results)
    return pd.DataFrame(data_list)

df = parse_data(all_data)

# 保存为 Excel 文件
file_path = '/content/drive/My Drive/horse_data2.xlsx'
df.to_excel(file_path, index=False)

print(f"数据已保存至 {file_path}")


Scraping data for 05/06/2024
Scraping data for 02/06/2024
Scraping data for 01/06/2024
Scraping data for 29/05/2024
Scraping data for 26/05/2024
Scraping data for 22/05/2024
Scraping data for 19/05/2024
Scraping data for 18/05/2024
Scraping data for 15/05/2024
Scraping data for 12/05/2024
Scraping data for 11/05/2024
Scraping data for 08/05/2024
Scraping data for 05/05/2024
Scraping data for 04/05/2024
Scraping data for 01/05/2024
Scraping data for 28/04/2024
Scraping data for 24/04/2024
Scraping data for 20/04/2024
Scraping data for 17/04/2024
Scraping data for 14/04/2024
Scraping data for 13/04/2024
Scraping data for 10/04/2024
Scraping data for 07/04/2024
Scraping data for 06/04/2024
Scraping data for 03/04/2024
Scraping data for 31/03/2024
Scraping data for 30/03/2024
Scraping data for 27/03/2024
Scraping data for 24/03/2024
Scraping data for 20/03/2024
Scraping data for 16/03/2024
Scraping data for 13/03/2024
Scraping data for 10/03/2024
Scraping data for 06/03/2024
Scraping data 

In [35]:
# 讀取數據
file_path = '/content/drive/My Drive/horse_data2.xlsx'
df = pd.read_excel(file_path)

# 先過濾掉 NaN 或非字符串的值
df = df.dropna(subset=['賽事日期和地點'])
df['賽事日期和地點'] = df['賽事日期和地點'].astype(str)


# 只保留 '賽事日期和地點' 列中的 '跑馬地' 或 '沙田' 字符串
df['賽事日期和地點'] = df['賽事日期和地點'].apply(lambda x: '跑馬地' if '跑馬地' in x else '沙田' if '沙田' in x else None)

filtered_df = df.dropna()
filtered_file_path = '/content/drive/My Drive/horse_data_filtered.xlsx'
filtered_df.to_excel(filtered_file_path, index=False)

print(f"過濾後的數據已保存至 {filtered_file_path}")

過濾後的數據已保存至 /content/drive/My Drive/horse_data_filtered.xlsx


In [ ]:
filtered_file_path = '/content/drive/My Drive/horse_data_filtered.xlsx'
filtered_df = pd.read_excel(filtered_file_path)
display(filtered_df)
filtered_file_path2 = '/content/drive/My Drive/racing_course_info.xlsx'
filtered_df2 = pd.read_excel(filtered_file_path2)
display(filtered_df2)
file_path = '/content/drive/My Drive/horse_data.xlsx'
racing_course_info = pd.read_excel(file_path)
display(racing_course_info)

In [36]:
import pandas as pd

# Ensure the date columns are in the same format
horse_data_filtered['date'] = pd.to_datetime(horse_data_filtered['date'], dayfirst=True)
horse_data['date'] = pd.to_datetime(horse_data['date'], dayfirst=True)

# Correctly rename the column in horse_data_filtered if necessary
horse_data_filtered.rename(columns={'賽事日期和地點': '地點'}, inplace=True)

# Merge the two files based on the date
merged_data = pd.merge(horse_data, horse_data_filtered[['date', '地點', '場地狀況', '賽道']], on='date', how='left')

# Save the merged data to a new Excel file
output_path = '/content/drive/My Drive/merged_horse_data.xlsx'
merged_data.to_excel(output_path, index=False)

In [40]:
display(merged_data)

,馬號,實際負磅,排位體重,檔位,名次,騎師_0,騎師_1,騎師_2,騎師_3,騎師_4,...,練馬師_15,練馬師_16,練馬師_17,練馬師_18,練馬師_19,練馬師_20,練馬師_21,練馬師_22,練馬師_23,race_id
0,4.0,122,1153,1,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,8.0,116,1156,3,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,1.0,135,1034,7,3.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,3.0,127,1047,6,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,7.0,115,1062,2,5.0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,8.0,126,1445,12,10.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1960
1960,5.0,125,1183,3,11.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1961
1961,10.0,127,1011,14,12.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1962
1962,7.0,128,1066,1,13.0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1963


In [37]:
# Step 3: Filter the data based on '名次'
def clean_ranking(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, str) and '平頭馬' in x:
        return int(x.replace('平頭馬', ''))
    try:
        return int(x)
    except ValueError:
        return np.nan

if '名次' in merged_data.columns:
    merged_data['名次'] = merged_data['名次'].apply(clean_ranking)
else:
    print("Column '名次' does not exist in the DataFrame.")

# Step 4: Select relevant columns and drop NaN values
relevant_columns = ['馬號', '騎師', '練馬師', '實際負磅', '排位體重', '檔位', '名次']
merged_data = merged_data[relevant_columns].dropna()

# Step 5: Filter out rows with non-numeric '檔位' values and convert to integers
merged_data = merged_data[merged_data['檔位'].apply(lambda x: str(x).isdigit())]
merged_data['檔位'] = merged_data['檔位'].astype(int)

# Step 6: Filter out strange draw values (e.g., negative values, or values beyond a realistic range)
merged_data = merged_data[(merged_data['檔位'] > 0) & (merged_data['檔位'] <= 20)]  # Assuming realistic range is 1 to 20

# Step 7: Encode categorical columns
label_encoder_jockey = LabelEncoder()
label_encoder_trainer = LabelEncoder()

merged_data['騎師'] = label_encoder_jockey.fit_transform(merged_data['騎師'])
merged_data['練馬師'] = label_encoder_trainer.fit_transform(merged_data['練馬師'])

merged_data = pd.get_dummies(merged_data, columns=['騎師', '練馬師'])

# Ensure boolean columns are converted to int
for col in merged_data.select_dtypes(include=['bool']).columns:
    merged_data[col] = merged_data[col].astype(int)

# Step 8: Add a new column 'race_id' based on existing columns
merged_data['race_id'] = merged_data.index + 1

display(merged_data) # Display the first few rows of the processed DataFrame

,馬號,實際負磅,排位體重,檔位,名次,騎師_0,騎師_1,騎師_2,騎師_3,騎師_4,...,練馬師_15,練馬師_16,練馬師_17,練馬師_18,練馬師_19,練馬師_20,練馬師_21,練馬師_22,練馬師_23,race_id
0,4.0,122,1153,1,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,8.0,116,1156,3,2.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,1.0,135,1034,7,3.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,3.0,127,1047,6,4.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
4,7.0,115,1062,2,5.0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959,8.0,126,1445,12,10.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1960
1960,5.0,125,1183,3,11.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1961
1961,10.0,127,1011,14,12.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1962
1962,7.0,128,1066,1,13.0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1963


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error


In [ ]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
# 特徵工程
def feature_engineering(df):
    df['排位體重'] = pd.to_numeric(df['排位體重'], errors='coerce')
    df['實際負磅'] = pd.to_numeric(df['實際負磅'], errors='coerce')

    # 新的特徵創建，例如馬匹的平均速度
    df['average_speed'] = df['排位體重'] / df['實際負磅']
    return df

df = feature_engineering(df)

# 處理缺失值
df = df.dropna()

# 定義特徵和目標
features = df.drop(columns=['名次', '馬號', 'race_id'])
target = df['名次']

# 特徵縮放
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# 切分數據集
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# 定義回調函數（如早停）
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 建立神經網絡模型
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=512, step=32), input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_3', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_4', min_value=32, max_value=64, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='linear'))  # 使用線性激活進行迴歸

    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1]))
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# 使用 Keras Tuner 進行超參數調整
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='intro_to_kt'
)

tuner.search(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[early_stopping])

# 獲取最佳模型
best_model = tuner.get_best_models(num_models=1)[0]

# 訓練最佳模型
history = best_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# 評估模型
loss, mae = best_model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')

Trial 10 Complete [00h 00m 16s]
val_loss: 13.364341735839844

Best val_loss So Far: 12.320350646972656
Total elapsed time: 00h 01m 46s


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 12.1666 - mae: 2.8210 - val_loss: 14.2951 - val_mae: 3.0167
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.0686 - mae: 2.6563 - val_loss: 12.9937 - val_mae: 2.9953
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 11.5232 - mae: 2.7809 - val_loss: 14.2152 - val_mae: 3.0334
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 10.8542 - mae: 2.6990 - val_loss: 14.1077 - val_mae: 3.0166
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.4216 - mae: 2.7318 - val_loss: 14.8849 - val_mae: 3.2360
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 11.2375 - mae: 2.7140 - val_loss: 14.2940 - val_mae: 3.0526
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 12.3008 - mae: 2.8030 - val_loss: 13.8519 - val_mae: 3.0309
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.5868 - mae: 2.6838 - val_loss: 12.4493 - val_mae: 2.9056
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/s

# **Step 2: Define a Function to Predict Top 4 Horses**

In [ ]:
def predict_top_4(model, scaler, label_encoder_jockey, label_encoder_trainer, input_data):
    input_df = pd.DataFrame(input_data, columns=['馬號', '馬名', '檔位', '負磅', '騎師', '練馬師', '馬匹體重'])

    # Preserve '馬號' and '馬名' columns
    horse_ids = input_df['馬號']
    horse_names = input_df['馬名']

    # Encode categorical features
    input_df['騎師'] = label_encoder_jockey.transform(input_df['騎師'])
    input_df['練馬師'] = label_encoder_trainer.transform(input_df['練馬師'])
    input_df = pd.get_dummies(input_df, columns=['騎師', '練馬師'])

    # Reindex to match the training feature set, excluding '馬號' and '馬名'
    feature_columns = [col for col in features.columns if col not in ['馬號', '馬名']]
    input_df = input_df.reindex(columns=feature_columns, fill_value=0)

    # Normalize the input features
    input_scaled = scaler.transform(input_df)

    # Make predictions
    predictions = model.predict(input_scaled)
    input_df['prediction'] = predictions

    # Add back '馬號' and '馬名' columns
    input_df['馬號'] = horse_ids
    input_df['馬名'] = horse_names

    # Get the top 4 horses with the highest probabilities of winning
    top_4_horses = input_df.nlargest(4, 'prediction')
    return top_4_horses[['馬號', '馬名', 'prediction']]



#Input Data then Predict

In [ ]:

# Example input data
input_data = [
    [1, '勁無敵', 5, 135, '蔡明紹','羅富全','1039'],
    [2, '同滿滿', 10, 135, '霍宏聲','伍鵬志','1082'],
    [3, '會展二號', 4, 134, '鍾易禮','告東尼','1145'],
    [4, '威武良駒', 7, 133, '布文','姚本輝',''],
    [5,'中華威威',8,132,'黃智弘','徐雨石','1243'],
    [6,'有你有我',12,130, '艾道拿','廖康銘','973'],
    [7,'飛躍精英',3,129, '巴度','賀賢','1088'],
    [8,'佳景臨門',1,128, '艾兆禮','葉楚航','1119'],
    [9,'萬事有',6,124,'希威森','容天鵬','1242'],
    [10,'友誼至佳',9,119,'董明朗','韋達','1182'],
    [11,'月球',11,118,'湯普新','黎昭昇','1154'],
    [12,'錢途光明',2,115,'潘明輝','丁冠豪','1068']
]

top_4_horses = predict_top_4(best_model, scaler, label_encoder_jockey, label_encoder_trainer, input_data)
display(top_4_horses)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


,馬號,馬名,prediction
2,3,會展二號,7.193602
11,12,錢途光明,7.079595
8,9,萬事有,6.862524
10,11,月球,6.730217
